# **IMPORT LIBRARY**

In [1]:
!pip install pytorch_lightning==1.7.0
!pip install quaterion

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.9/700.9 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 KB 7.0 MB/s eta 0:00:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pipinyulianto14","key":"bc31010ad7a199bebb04b6d33dfe572e"}'}

In [4]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 71 Mar  9 12:48 kaggle.json


In [5]:
!pip install -q kaggle

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!pwd

/content


In [9]:
!kaggle datasets download -d ispritchin/food-4-pizza-risotto-steak-sushi

100% 196M/196M [00:08<00:00, 32.4MB/s]
100% 196M/196M [00:08<00:00, 25.1MB/s]


In [10]:
!unzip 'food-4-pizza-risotto-steak-sushi.zip' -d './dataset'

Archive:  food-4-pizza-risotto-steak-sushi.zip
  inflating: ./dataset/food-4/test/pizza/2667824.jpg  
  inflating: ./dataset/food-4/test/pizza/2670730.jpg  
  inflating: ./dataset/food-4/test/pizza/2671508.jpg  
  inflating: ./dataset/food-4/test/pizza/2674351.jpg  
  inflating: ./dataset/food-4/test/pizza/2687575.jpg  
  inflating: ./dataset/food-4/test/pizza/2693334.jpg  
  inflating: ./dataset/food-4/test/pizza/2694223.jpg  
  inflating: ./dataset/food-4/test/pizza/2697971.jpg  
  inflating: ./dataset/food-4/test/pizza/2700543.jpg  
  inflating: ./dataset/food-4/test/pizza/2702825.jpg  
  inflating: ./dataset/food-4/test/pizza/2705497.jpg  
  inflating: ./dataset/food-4/test/pizza/2707814.jpg  
  inflating: ./dataset/food-4/test/pizza/2711828.jpg  
  inflating: ./dataset/food-4/test/pizza/2719697.jpg  
  inflating: ./dataset/food-4/test/pizza/2722646.jpg  
  inflating: ./dataset/food-4/test/pizza/2723529.jpg  
  inflating: ./dataset/food-4/test/pizza/2739039.jpg  
  inflating: ./dat

# **DATA PREPARATION**

In [11]:
import os
import tqdm
import torch
import pickle
import numpy as np
import torch.nn as nn
import torchvision
import shutil

from quaterion import Quaterion
from torch import nn
from typing import Callable
from quaterion_models.encoders import Encoder
from typing import Dict, Optional, Union
from quaterion import TrainableModel
from quaterion.eval.attached_metric import AttachedMetric
from quaterion.eval.group import RetrievalRPrecision
from quaterion.loss import SimilarityLoss, TripletLoss
from quaterion.train.cache import CacheConfig, CacheType
from pytorch_lightning import seed_everything
from torch.utils.data import Dataset, Subset
from torchvision import datasets, transforms
from quaterion_models.heads import EncoderHead, SkipConnectionHead
from pytorch_lightning.callbacks import EarlyStopping, ModelSummary
from quaterion.dataset import GroupSimilarityDataLoader, SimilarityGroupSample

# **CONFIGURATION**

In [12]:
DATASET = os.path.join("./dataset/food-4")
TRAIN_PATH = os.path.join("./dataset/food-4/train")
TEST_PATH = os.path.join("./dataset/food-4/test")

#Hyperparameter
NUM_EPOCHS = 50
LEARNING_RATE = 1e-3
TRAIN_BATCH_SIZE = 32
IMAGE_SIZE = 336

# LOAD **DATASET**

In [13]:
class foodDataset(Dataset):
  def __init__(self, dataset: Dataset, transform: Callable):
    self._dataset = dataset
    self._transform = transform

  def __len__(self) -> int:
    return len(self._dataset)

  def __getitem__(self, index) -> SimilarityGroupSample:
    image, label = self._dataset[index]
    image = self._transform(image)

    return SimilarityGroupSample(obj=image, group=label)

def get_datasets(
    input_size: int,
    split_cache_path="split_cache.pkl",
):

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    train_transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])

    test_transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])

    trainset = datasets.ImageFolder(TRAIN_PATH)
    testset = datasets.ImageFolder(TEST_PATH)
    full_dataset = trainset + testset

    train_indices, test_indices = None, None

    if not split_cache_path or not os.path.exists(split_cache_path):

        train_categories = np.random.choice(a=196, size=196 // 2, replace=False)

        labels_list = np.array([label for _, label in tqdm.tqdm(full_dataset)])

        labels_mask = np.isin(labels_list, train_categories)

        train_indices = np.argwhere(labels_mask).squeeze()

        test_indices = np.argwhere(np.logical_not(labels_mask)).squeeze()

    if train_indices is None or test_indices is None:
        train_indices, test_indices = pickle.load(open(split_cache_path, "rb"))
    else:
        pickle.dump((train_indices, test_indices), open(split_cache_path, "wb"))

    train_dataset = foodDataset(
        Subset(full_dataset, train_indices), transform=train_transform
    )

    test_dataset = foodDataset(
        Subset(full_dataset, test_indices), transform=test_transform
    )

    return train_dataset, test_dataset


def get_dataloaders(
    batch_size: int,
    input_size: int,
    shuffle: bool = False,
    split_cache_path="split_cache.pkl",
):
    train_dataset, test_dataset = get_datasets(input_size, split_cache_path)

    train_dataloader = GroupSimilarityDataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle)

    test_dataloader = GroupSimilarityDataLoader(
        test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

# **FEATURE VECTOR ENCODER**

In [14]:
class foodEncoder(Encoder):
    def __init__(self, encoder_model: nn.Module):
        super().__init__()
        self._encoder = encoder_model
        self._embedding_size = 2048  # last dimension from the ResNet model

    @property
    def trainable(self) -> bool:
        return False

    @property
    def embedding_size(self) -> int:
        return self._embedding_size

    def forward(self, images):
        embeddings = self._encoder.forward(images)
        return embeddings

    def save(self, output_path: str):
        os.makedirs(output_path, exist_ok=True)
        torch.save(self._encoder, os.path.join(output_path, "encoder.pth"))

    @classmethod
    def load(cls, input_path):
        encoder_model = torch.load(os.path.join(input_path, "encoder.pth"))
        return foodEncoder(encoder_model)

# **DECODER**

In [15]:
class Model(TrainableModel):
    def __init__(self, lr: float, mining: str):
        self._lr = lr
        self._mining = mining
        super().__init__()

    def configure_encoders(self) -> Union[Encoder, Dict[str, Encoder]]:
        pre_trained_encoder = torchvision.models.resnet152(weights='IMAGENET1K_V1')
        pre_trained_encoder.fc = nn.Identity()
        return foodEncoder(pre_trained_encoder)

    def configure_head(self, input_embedding_size) -> EncoderHead:
        return SkipConnectionHead(input_embedding_size, dropout=0.1)

    def configure_loss(self) -> SimilarityLoss:
        return TripletLoss(mining=self._mining, margin=0.5)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), self._lr)
        return optimizer

    def configure_caches(self) -> Optional[CacheConfig]:
        return CacheConfig(
            cache_type=CacheType.AUTO, save_dir="./cache_dir", batch_size=32
        )

    def configure_metrics(self) -> AttachedMetric:
        return AttachedMetric(
            "rrp",
            metric=RetrievalRPrecision(),
            prog_bar=True,
            on_epoch=True,
            on_step=False,
        )

In [16]:
def train(
    lr: float,
    mining: str,
    batch_size: int,
    epochs: int,
    input_size: int,
    shuffle: bool,
    save_dir: str,
):

    model = Model(
        lr=lr,
        mining=mining,
    )
    import warnings

    warnings.filterwarnings("ignore", ".*does not have many workers.*")

    train_dataloader, val_dataloader = get_dataloaders(
        batch_size=batch_size, input_size=input_size, shuffle=shuffle
    )

    Quaterion.fit(
        trainable_model=model,
        trainer=None,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
    )

    shutil.rmtree(save_dir, ignore_errors=True)
    model.save_servable(save_dir)


In [17]:
train(lr=LEARNING_RATE, 
      mining="hard", 
      batch_size=TRAIN_BATCH_SIZE, 
      epochs=NUM_EPOCHS, 
      input_size=IMAGE_SIZE, 
      shuffle=True, save_dir="foods_recognition"
      )

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

100%|██████████| 4000/4000 [00:18<00:00, 221.10it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/base.py:22: LightningDeprecationWarning: pytorch_lightning.callbacks.base.Callback has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.callbacks.callback.Callback class instead.
  rank_zero_deprecation(
2023-03-09 12:49:05.918 | DEBUG    | quaterion.train.cache_mixin:_cache:168 - Using full cache
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

2023-03-09 12:50:19.419 | DEBUG    | quaterion.train.cache_mixin:_cache:223 - Caching has been successfully finished
2023-03-09 12:50:21.746 | DEBUG    | quaterion.train.cache_mixin:save_cache:385 - Cache saved to ./cache_dir
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                     ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _model                   │ SimilarityModel    │ 62.3 M │
│ 1 │ _model.default           │ foodEncoder        │ 58.1 M │
│ 2 │ _model.default._encoder  │ ResNet             │ 58.1 M │
│ 3 │ _model.head              │ SkipConnectionHead │  4.2 M │
│ 4 │ _model.head.dropout      │ Dropout            │      0 │
│ 5 │ _model.head.fc_0         │ Linear             │  4.2 M │
│ 6 │ _model.head.skip_dropout │ Identity           │      0 │
│ 7 │ _loss                    │ TripletLoss        │      0 │
└───┴──────────────────────────┴────────────────────┴────────┘

Trainable params: 4.2 M                                                                                            
Non-trainable params: 58.1 M                                                                                       
Total params: 62.3 M                                                                                               
Total estimated model params size (MB): 249

Output()

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:1972: LightningDeprecationWarning: `Trainer.reset_train_val_dataloaders` has been deprecated in v1.7 and will be removed in v1.9. Use `Trainer.reset_{train,val}_dataloader` instead
  rank_zero_deprecation(
